In [1]:
import datetime
import os

import backtrader
import numpy as np

os.chdir("../src")

from predictor import lstm

ModuleNotFoundError: No module named 'lstm'

In [2]:
# datafeed to provide indexes to frames to be used by a strategy
class FrameFeed(backtrader.feed.DataBase):
    params = (
        ("fromdate", datetime.datetime),
        ("todate", datetime.datetime),
        ("frames", None),
    )

    lines = ("datetime", "frame_index", "close")

    def __init__(self):
        super()

    def start(self):
        super(backtrader.feed.DataBase, self).start()
        # TODO: use assignment expressions
        self.frames = (
            frame for frame in enumerate(self.p.frames)
            if self.p.fromdate <= frame[1][-1][-1][0] <= self.p.todate
        )

    def _load(self):
        try:
            i, frame = next(self.frames)
        except StopIteration:
            return False

        self.lines.datetime[0] = backtrader.utils.date2num(frame[-1][-1][0])
        self.lines.frame_index[0] = i

        return True

In [3]:
# strategy to use by backtesting
class TestStrategy(backtrader.Strategy):
    params = (
        ("model", None),
        ("frames", [[[]]])
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt, txt))

    def __init__(self):
        self.closings = self.datas[0].close
        self.frame_index = self.datas[1].frame_index
        self.frames = self.params.frames
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        
        print(f"order size: {order.size}")
        
#         new_s, r, done, _ = env.step(a)
#         q_table[s, a] += r + lr * (y * np.max(q_table[new_s, :]) - q_table[s, a])
#         s = new_s

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED:')

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log("SELL EXECUTED:")

            print(f"Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}")
            print(self.broker.getvalue())

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def next(self):
        if self.order:
            return

        current_frame = [[self.closings[i]] for i in range(0, -15, -1)]
        previous_frame = [[self.closings[i]] for i in range(-1, -16, -1)]

        current_normalized_frame = lstm.normalize_frame(current_frame)
        previous_normalized_frame = lstm.normalize_frame(previous_frame)
    
        current_prediction = lstm.predict_sequences_multiple(self.p.model, [current_normalized_frame])
        previous_prediction = lstm.predict_sequences_multiple(self.p.model, [previous_normalized_frame])
        
        current_prediction_denorm = lstm.denormalize_dim(current_prediction[0][0], current_frame[0][0])
        # previous_prediction_denorm = lstm.denormalize_dim(previous_prediction[0][0], previous_frame[0][0])
        
        # print(f"current: {current_frame}, prediction: {current_prediction_denorm}")
        
        pred_diff = current_prediction[0][0] - previous_prediction[0][0]
        share_price = current_frame[-1][0]
        
        if not self.position:
            # can only hold, buy, or short
            if pred_diff > 0:
                if current_prediction_denorm > share_price:
                    self.log("BUY")
                    self.order = self.buy()
                else:
                    self.log("SHORT")
                    self.order = self.sell()
        else:
            # can only close or hold
            self.log("CLOSING")
            self.order = self.close()
        

#         if pred_diff > 0 and current_prediction_denorm > share_price:
#             if not self.position:
#                 self.log("BUY")
#                 self.order = self.buy()
                
#         elif pred_diff < 0 and current_prediction_denorm < share_price:
#             if not self.position:
#                 self.log("SHORT")
#                 self.order = self.sell()
                
#         elif self.position:
#             self.log("CLOSING")
#             self.order = self.close()
            
            
                

In [4]:
class Reverser(backtrader.sizers.FixedSize):

    def _getsizing(self, comminfo, cash, data, isbuy):
        position = self.broker.getposition(data)
        return cash // data.open[0] - 1
#         size = self.p.stake * (1 + (position.size != 0))
#         return size
#         if isbuy:
#             return cash // data.open[0] - 1
#         else:
#             return position.size

In [15]:
SYMBOL="XSD"

# get testing data
times = lstm.get_time_series_daily(SYMBOL, ["1. open"], outputsize="full")
vectors = lstm.times_to_vectors(times, include_time=True)[::-1]

train_vectors, test_vectors = lstm.partition_data(vectors, partition_coefficient=0.8)

train_frames = lstm.get_frames(train_vectors, 15, with_target=True)
test_frames = lstm.get_frames(test_vectors, 15, with_target=False)

train_no_dates = [[[col[1] for col in vector] for vector in frame] for frame in train_frames]
normalized_train = lstm.normalize_frames(train_no_dates)
x_train, y_train = lstm.seperate_xy(normalized_train)

In [16]:
# setup model
model = lstm.setup_lstm_model(x_train, y_train)

compilation time :  0.019165754318237305
Train on 2536 samples, validate on 134 samples
Epoch 1/1
2536/2536 [==============================] - 2s 823us/step - loss: 0.1416 - val_loss: 0.0756


In [17]:
# setup inital testing strategy
cerebro = backtrader.Cerebro()
cerebro.broker.setcash(100000.0)
cerebro.addsizer(Reverser)
cerebro.broker.setcommission(commission=0.001)

# add data and model to strategy
cerebro.addstrategy(TestStrategy, model=model, frames=test_frames)

from_date = datetime.datetime(2019, 4, 1)
to_date = datetime.datetime(2019, 6, 1)

cerebro.adddata(
    backtrader.feeds.YahooFinanceData(
        dataname=SYMBOL,
        fromdate=from_date,
        todate=to_date
    )
)

cerebro.adddata(
    FrameFeed(
        frames=test_frames,
        fromdate=from_date,
        todate=to_date
    )
)

print(f'Beginning Portfolio Value: {cerebro.broker.getvalue()}')

cerebro.run()

print(f'Final Portfolio Value: {cerebro.broker.getvalue()}')

Beginning Portfolio Value: 100000.0
2019-04-04, SHORT
order size: -1210.0
2019-04-05, SELL EXECUTED:
Price: 83.3, Cost: -100793.0, Comm: 100.79299999999999
99741.90699999999
2019-04-05, CLOSING
order size: 1210.0
2019-04-08, BUY EXECUTED:
Price: 83.1, Cost: -100793.0, Comm: 100.55099999999999
100040.65599999999
2019-04-08, BUY
order size: 1202.0
2019-04-08, Order Canceled/Margin/Rejected
2019-04-08, BUY
order size: 1202.0
2019-04-09, Order Canceled/Margin/Rejected
2019-04-09, BUY
order size: 1199.0
2019-04-10, BUY EXECUTED:
Price: 82.92, Cost: 99421.08, Comm: 99.42108
101535.90491999999
2019-04-10, CLOSING
order size: -1199.0
2019-04-11, SELL EXECUTED:
Price: 84.34, Cost: 99421.08, Comm: 101.12366000000002
101542.69125999999
2019-04-11, BUY
order size: 1202.0
2019-04-12, Order Canceled/Margin/Rejected
2019-04-15, BUY
order size: 1188.0
2019-04-16, BUY EXECUTED:
Price: 85.25, Cost: 101277.0, Comm: 101.277
103425.37426
2019-04-16, CLOSING
order size: -1188.0
2019-04-17, SELL EXECUTED:
Pr